In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import re
import pandas as pd

In [2]:
# WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 스크롤 로직
def scroll_to_bottom(scrollable_element):
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
    while True:
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)
        time.sleep(1)
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
        if new_height == last_height:
            break
        last_height = new_height

# 네이버 지도 접속
driver.get("https://map.naver.com/")
time.sleep(3)

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
)
search_box.send_keys("송파구 맛집")
search_box.send_keys(Keys.ENTER)
time.sleep(5)

# iframe 전환
try:
    iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
    )
    driver.switch_to.frame(iframe)
except Exception as e:
    print(f"iframe 전환 오류: {e}")
    driver.quit()
    exit()

# 결과 리스트 초기화
results = []
visited_urls = set()
page_number = 1
MAX_PAGES = 5  # 최대 페이지 수
MAX_RESULTS = 150  # 최대 결과 수 제한

while page_number <= MAX_PAGES and len(results) < MAX_RESULTS:
    try:
        # 스크롤을 맨 아래까지 내리기
        scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
        scroll_to_bottom(scrollable_element)

        # 현재 페이지의 가게 리스트 가져오기
        store_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "place_bluelink"))
        )
        print(f"현재 {page_number}페이지) 총 {len(store_elements)}개의 가게를 찾았습니다.")

        for store in store_elements:
            if len(results) >= MAX_RESULTS:
                print("최대 결과 수에 도달하여 크롤링을 종료합니다.")
                break

            try:
                store.click()
                time.sleep(random.uniform(3, 5))

                # 상세 iframe 전환
                driver.switch_to.default_content()
                detail_iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe"))
                )
                driver.switch_to.frame(detail_iframe)

                # 페이지 소스 파싱
                html = driver.page_source
                soup = BeautifulSoup(html, "lxml")

                # 가게 정보 추출
                place_name = soup.select_one("span.GHAhO").text.strip() if soup.select_one("span.GHAhO") else "N/A"
                category = soup.select_one("span.lnJFt").text.strip() if soup.select_one("span.lnJFt") else "N/A"
                address = soup.select_one("span.LDgIH").text.strip() if soup.select_one("span.LDgIH") else "N/A"
                current_url = driver.current_url
                res_code = re.findall(r"place/(\d+)", current_url)
                store_url = f'https://pcmap.place.naver.com/restaurant/{res_code[0]}/review/visitor#' if res_code else "N/A"

                # 중복 여부 확인
                if store_url not in visited_urls:
                    visited_urls.add(store_url)
                    results.append({
                        "가게 이름": place_name,
                        "업종구분": category,
                        "주소": address,
                        "URL": store_url
                    })
                    print(f"{len(results)}. {place_name}")

                # iframe 복귀
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"가게 크롤링 중 오류 발생: {e}")

        if len(results) >= MAX_RESULTS:
            print("최대 결과 수에 도달하여 크롤링을 종료합니다.")
            break

        # '다음' 버튼 클릭
        if page_number < MAX_PAGES:
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//a[@class="eUTV2" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", next_button)
                print(f"'다음' 버튼 클릭: 페이지 {page_number + 1}로 이동 중")
                time.sleep(random.uniform(5, 10))

                # iframe 전환
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"'다음' 버튼 클릭 중 오류 발생: {e}")
                break

        # 페이지 번호 증가
        page_number += 1

    except Exception as e:
        print(f"페이지 크롤링 중 오류 발생: {e}")
        break

file_name = "Songpa-gu_restaurants.csv"
df = pd.DataFrame(results)
df.to_csv(file_name, index=False, encoding="utf-8-sig")

print(f"크롤링 완료. 결과는 {file_name}에 저장되었습니다.")
driver.quit()

현재 1페이지) 총 76개의 가게를 찾았습니다.
1. 크래버대게나라 잠실점
2. 갯벌의 조개 잠실새내점
3. 프라자손칼국수
4. 일월고기
5. 요리하는남자
6. 키친205 롯데월드몰점
7. 팻어케이크
8. 신천돼지 잠실본점
9. 부농정육식당
10. 래빗캐슬
11. 씨엘트리
12. 모미지 잠실방이점
13. 개나리아구찜 본점
14. 밸런티 송파 본점
15. 크래프트아일랜드 잠실점
16. 잠실장어와 한우
17. 진지아
18. 비채나
19. 피자네버슬립스 송리단길점
20. 아리산
21. 차만다
22. 간코 롯데월드몰점
23. 천상애장어가 가락본점
24. 도몽 잠실새내 본점
25. 만옥 롯데월드몰점
26. 숯불성화
27. 별미곱창
28. 갓잇 송리단길점
29. 각광 잠실본점
30. 가락시장 싱싱해수산
31. 르빵 송파점
32. 한성백제얼큰샤브칼국수 올림픽공원본점
33. 수작나베 석촌점
34. VGG
35. 바이킹스워프 롯데월드몰점
36. 배키욘방
37. 함경도찹쌀순대
38. 홈수끼
39. 완도산회포장마차 송파직영점
40. 치즈룸&테이스팅룸 롯데월드몰점
41. 어부바
42. 육화식당
43. 청와옥 석촌호수직영점
44. MIP 잠실석촌호수본점
45. 방이화로
46. 삼정집 석촌호수점
47. 김태완스시 송파본점
48. 일월껍데기
49. 시그니엘 서울 STAY
50. 가락시장 나래수산
51. 파도집
52. 스시이세이 잠실 롯데백화점 에비뉴엘 월드타워점
53. 바다약방 잠실새내점
54. 카페아일랜드
55. 청와옥 본점
56. 해닭 올림픽공원점
57. 대성정육식당 잠실새내역본점
58. 어거스트힐 방이점
59. 대현굴국밥
60. 토마루해물칼국수
61. 카페 페퍼
62. 호낙가 본점
63. 진곱창
64. 빽돈 방이점
65. 매드포갈릭 롯데시네마월드타워점
66. 디저티스트
67. 진대감 방이점
68. 디에스티클럽
69. 만족오향족발 잠실신천점
70. 헝그리홀스
71. 롯데호텔월드 라세느
72. 옛촌매운갈비찜
73. 신천생태찌개
74. 몽중헌 방이점
75. 한우상회
76. 골드참치
'다